# Credit Risk Resampling Techniques

In [38]:
import warnings
warnings.filterwarnings('ignore')

In [39]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Read the CSV into DataFrame

In [40]:
# Load the data
file_path = Path('lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [41]:
df["homeowner"].value_counts()


mortgage    38572
own         30930
rent         8034
Name: homeowner, dtype: int64

# Split the Data into Training and Testing

In [42]:
# 
# Create our features
X =  df.drop(columns=['loan_status'])

# Create our target
y = df['loan_status']

In [43]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [44]:
# Check the balance of our target values
# This is an imbalanced dataset
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [45]:
# encode homeowner column and target data
X=pd.get_dummies(data=X, columns=['homeowner'], drop_first=True)
X

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0,0
...,...,...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600,1,0
77532,17700.0,10.662,80900,0.629172,11,2,50900,0,0
77533,17600.0,10.595,80300,0.626401,11,2,50300,0,1
77534,16300.0,10.068,75300,0.601594,10,2,45300,0,0


In [46]:
# Utilize label encoding for target data
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
le = preprocessing.LabelEncoder()
le.fit(y)
LabelEncoder()
y=le.transform(y)


In [47]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [48]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

In [49]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaler = scaler.fit(X)
X = X_scaler.transform(X)

In [50]:
# Making sure X is scaled
X

array([[ 0.42730427,  0.4268375 ,  0.42740435, ...,  0.42740435,
         1.22752672, -0.33999088],
       [-0.67148676, -0.67491817, -0.67155173, ..., -0.67155173,
         1.22752672, -0.33999088],
       [-0.38484562, -0.370249  , -0.37292236, ..., -0.37292236,
        -0.81464622,  2.9412554 ],
       ...,
       [ 3.72367737,  3.71299241,  3.71232744, ...,  3.71232744,
        -0.81464622,  2.9412554 ],
       [ 3.10262157,  3.12051768,  3.1150687 , ...,  3.1150687 ,
        -0.81464622, -0.33999088],
       [ 2.76820691,  2.75401528,  2.75671345, ...,  2.75671345,
        -0.81464622, -0.33999088]])

In [51]:
X_scaler = scaler.fit(X_train)

In [52]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

# Simple Logistic Regression

In [53]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [54]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9892813049736127

In [55]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  615,    10],
       [  102, 18657]], dtype=int64)

In [56]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.98      0.99      0.92      0.99      0.98       625
          1       1.00      0.99      0.98      1.00      0.99      0.98     18759

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [57]:
# Resample the training data with the RandomOversampler
# View the count of target classes with Counter
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({1: 56277, 0: 56277})

In [58]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [59]:
# Calculated the balanced accuracy score
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  111, 18648]], dtype=int64)

In [60]:
# Display the confusion matrix
balanced_accuracy_score(y_test, y_pred)

0.9946414201183431

In [61]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      1.00      0.99      0.92      0.99      0.99       625
          1       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [62]:
# Resample the training data with SMOTE
# View the count of target classes with Counter
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)
from collections import Counter

Counter(y_resampled)

Counter({1: 56277, 0: 56277})

In [63]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [64]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9946680739911509

In [65]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  110, 18649]], dtype=int64)

In [66]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      1.00      0.99      0.92      0.99      0.99       625
          1       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [67]:
# Resample the data using the ClusterCentroids resampler
# View the count of target classes with Counter
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 1875, 1: 1875})

In [68]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [69]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.9946680739911509

In [70]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  110, 18649]], dtype=int64)

In [71]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      1.00      0.99      0.92      0.99      0.99       625
          1       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [72]:
# Resample the training data with SMOTEENN
# View the count of target classes with Counter
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({0: 73923, 1: 74597})

In [74]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [75]:
# Calculate the balanced accuracy score
y_pred = model.predict(X_test)
BAS_SMTN = balanced_accuracy_score(y_test, y_pred)
BAS_SMTN

0.9945881123727278

In [76]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  113, 18646]], dtype=int64)

In [77]:
# Print the imbalanced classification report
cr_imb = classification_report_imbalanced(y_test, y_pred,output_dict=True)
recall_SMTN = cr_imb['avg_rec']
geo_SMTN = cr_imb['avg_geo']
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      1.00      0.99      0.91      0.99      0.99       625
          1       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

SMOTE had the best balanced accuracy

2. Which model had the best recall score?

They all had the same recall

3. Which model had the best geometric mean score?

combinaton oversampling and smote